# Przetwarzanie języka naturalnego w spaCy

## Instalacje

In [ ]:
!pip install -q spacy --upgrade

Musimy wybrać model językowy https://spacy.io/usage/models

In [ ]:
!python -m spacy download pl_core_news_md

In [ ]:
import spacy

nlp = spacy.load("pl_core_news_md")

Zmienna `nlp` zawiera średni model `pl_core_news_md` dla języka polskiego.


## Operacja na tekście


Przepuśćmy mały „dokument” przez parser języka naturalnego:

In [ ]:
text = "Litwo, Ojczyzno moja, Ty jesteś jak zdrowie, ile Cię trzeba cenić ten tylko się dowie, kto Cię stracił."
doc = nlp(text)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.is_stop)

Najpierw stworzyliśmy z tekstu [doc](https://:spacy.io/api/doc), który jest kontenerem na dokument i wszystkie jego adnotacje. Następnie przejrzeliśmy dokument, aby zobaczyć, co przeanalizował _spaCy_.


W tym prostym przypadku cały dokument to tylko jedno krótkie zdanie.
Dla każdego słowa w tym zdaniu _spaCy_ utworzył [token](https://spacy.io/api/token) i uzyskaliśmy dostęp do pól w każdym tokenie, aby pokazać:

  - surowy tekst
  - [lemma](https://en.wikipedia.org/wiki/Lemma_(morphology)) – rdzeń słowa
  - [część mowy](https://en.wikipedia.org/wiki/Część_mowy)
  - flaga określająca, czy słowo jest _stopword_ – czyli popularnym słowem, które można odfiltrować


Przeformatujmy parsowanie _spaCy_ tego zdania jako ramkę danych [pandas](https://pandas.pydata.org/):

In [ ]:
import pandas as pd

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)

df

Pandas to taki "pythonowy excel", kiedy użyjemy go do danych mamy pod ręką móstwo opcji. Możemy na przykład zrobić łatwo wykres jak często występują dane części mowy:

In [ ]:
df.POS.value_counts().plot(kind='bar')


Możemy dostać się do konkretnego słowa, jak jak w liście - poprzez podanie indeksu.

In [ ]:
doc[21].lemma_

Następnie użyjemy biblioteki [displaCy](https://ines.io/blog/developing-displacy) do wizualizacji drzewa analizy dla tego zdania:

In [ ]:
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

W dłuższych dokumentach musimy dzielić na zdania - potrzebujemy wykrywania granicy zdań — znanej również jako _segmentacja zdań_ — oparte na wbudowanym/domyślnym [sentencizer](https://spacy.io/api/sentencizer):

In [ ]:
text = "W mieścinie pewnej, prowincji Manchy, której nazwiska nie powiem, żył niedawnymi czasy hidalgo pewien, z liczby tych, co to prócz spisy u siodła, szabliska starego, szkapy chudziny i paru gończych, niewiele co więcej mają. Rosolina powszednia, z baraniny częściej niż z wołowiny wygotowana na obiad, bigosik z resztek obiadu prawie co wieczór na kolację, co piątek soczewica, co sobota jaja sadzone po hiszpańsku, a na niedzielę gołąbeczek jakiś w dodatku do codziennej strawy, zjadały mu corocznie trzy czwarte części całego dochodu. Reszta szła na przyodziewek: na opończę z sukna cienkiego, hajdawery aksamitne z takimiż łapciami i na świtkę z krajowego samodziału dobornego, którą się w powszednie dni tygodnia obchodził."

doc = nlp(text)

for sent in doc.sents:
    print(">", sent)

Gdy _spaCy_ tworzy dokument, wykorzystuje zasadę _nieniszczącej tokenizacji_, co oznacza, że tokeny, zdania itp. są po prostu indeksami w długiej tablicy. Innymi słowy, nie dzielą strumienia tekstu na małe kawałki. Tak więc każde zdanie to [span](https://spacy.io/api/span) z indeksem _start_ i _end_ w tablicy dokumentów:

In [ ]:
for sent in doc.sents:
    print(">", sent.start, sent.end)

Możemy zaindeksować tablicę dokumentów, aby wyciągnąć tokeny dla jednego zdania:

In [ ]:
doc[97:129]

## Natural Language Understanding



In [ ]:
text = "Józef Piłsudski herbu Piłsudski urodził się 5 grudnia 1867 w Zułowie na Wileńszczyźnie, w rodzinie o tradycjach patriotycznych."
doc = nlp(text)


In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_, ent.lemma_)

Biblioteka _displaCy_ zapewnia doskonały sposób wizualizacji nazwanych jednostek:

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

## Wektory

In [ ]:
import spacy

doc1 = nlp("Jedziemy na święta do domu.")
doc2 = nlp("Wracamy z wakacji nad morzem.")
doc3 = nlp("Uczę się na egzamin.")

print(doc1, "<->", doc2, doc1.similarity(doc2))
print(doc1, "<->", doc3, doc1.similarity(doc3))

wakacje = doc1[4]
dom = doc2[2]
print(wakacje, "<->", dom, wakacje.similarity(dom))

### Zadanie

1. Korzystając z API do wikipedii (pierwszy notatnik) pobrać artykuł na dowolny temat i korzystając ze spacy pobrać wszystkie osoby, które w nim występują (korzystając z lematow). Zrobić to analogicznie dla angielskiego (lematyzator do NER działa lepiej po angielsku). Wypisać wszystkie osoby w kolejnosci razem z informacjamą ile razy występują.

2. Wybrać dowolną krotką książkę z wolnych lektur. Korzystając z parsowania zależnosciowego wypisać wszystkie przymiotniki okreslające danego bohatera bezpośrednio (dep) albo okreslenie. Analogicznie dla angielskiego, przykład poniżej. Przyjrzeć się strukturze drzewa zależnosciowego i zaproponować ulepszenia - tak, żeby lepiej znajdować interesujące nas okreslenia.

In [ ]:
!wget https://wolnelektury.pl/media/book/txt/kamizelka.txt

In [ ]:
import re

In [ ]:
with open('kamizelka.txt', 'r') as file:
    data = file.read().replace('\n', '')

In [ ]:
doc = nlp(data)

In [ ]:
okreslenia = []
for sent in doc.sents:
    if re.search("kamizelk", str(sent)):
      for token in sent:
        print(token.text, token.lemma_, token.pos_,  token.dep_, token.head.text,)
        if re.search("kamizelk",token.head.text) and (token.dep_ == "amod" or token.dep_ == "nmod"):
          okreslenia.append(token)
      print("\n\n\n")

In [ ]:
print(okreslenia)